### 本章節包含以下內容：
* 第一個神經網路示例
* 張量與張量運算
* 神經網路如何通過反向傳播與梯度下降學習

### 關於類和標籤的說明：
在機器學習中，分類問題中的某個類別叫做類(class)，數據點叫做樣本(smaple)，某個樣本對應的類叫做標籤(label)。


## 2.1 初識神經網路
### 2-1 網路架構
神經網路的核心是層(layer)，層從輸入數據中提取**表示**(特徵)，而大多數的深度學習都是將簡單的層鏈接起來，從而實現漸進式的數據蒸餾( data distillation )。<br><br>

接下來，我們來演練基本的網路架構，此網路包含兩個Dense層，他們是密集連接(全連接)層，第二層(也是最後一層)是一個擁有10個輸出標籤的softmax層，並返回10個代表數字的機率值。其中，想要**訓練**神經網路的話，我們還需要選擇**編譯**步驟的三個參數：
* 損失函數( loss function )：衡量網路在「訓練數據」上的性能，即網路如何朝正確的方向前進。
* 優化器( Optimizer )：基於「訓練數據」和「損失函數」來更新網路的機制。
* 在「訓練」和「測試」過程中需要監控的指標( Metric )：本範例只考慮精度，及正確分類的圖像所佔的比例。


In [20]:
# 2-1 載入MNIST數據集
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print('train_image.shape = ', train_images.shape)
print('train_labels.shape = ', train_labels.shape)
print('test_image.shape = ', test_images.shape)
print('test_labels.shape = ', test_labels.shape)


train_image.shape =  (60000, 28, 28)
train_labels.shape =  (60000,)
test_image.shape =  (10000, 28, 28)
test_labels.shape =  (10000,)


In [21]:
# 2-2 網路架構
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))


In [22]:
# 2-3 編譯步驟
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])


In [23]:
# 2-4 準備圖像數據
'''
在開始訓練之前，我們要對數據進行預處理，將其轉變為網路所要求的形狀
亦即，縮放所有的輸入標籤都在[0,1]區間
'''
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255


In [24]:
# 2-5 準備標籤
'''
我們還需要對標籤進行影像分類(手動標上標籤)
第3章會對這一步進行解釋
'''
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


In [25]:
# 2-6 開始訓練神經網路
'''
現在我們準備開始訓練網路，在Keras中這一步是通過調用網路的fit方法來完成的
意思是，我們在訓練數據上 「擬和(fit)」模型
訓練過程中將顯示兩項資訊：
1.網路在「訓練數據」上的損失( loss )
2.網路在「訓練數據」上的準確度( acc )
'''
network.fit(train_images, train_labels, epochs=5, batch_size=128)


Epoch 1/5
60000/60000 [==============================] - 9s 147us/step - loss: 0.2584 - acc: 0.9244
Epoch 2/5
60000/60000 [==============================] - 8s 135us/step - loss: 0.1046 - acc: 0.9690
Epoch 3/5
60000/60000 [==============================] - 8s 141us/step - loss: 0.0696 - acc: 0.9793
Epoch 4/5
60000/60000 [==============================] - 9s 148us/step - loss: 0.0498 - acc: 0.9853
Epoch 5/5
60000/60000 [==============================] - 8s 140us/step - loss: 0.0369 - acc: 0.9888


In [26]:
# 2-7 檢查訓練資料於測試集的性能
test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_acc:', test_acc)


10000/10000 [==============================] - 1s 108us/step
test_acc: 0.9811


由上面結果可以得知，測試集準確度為97.8%，比訓練集準確度低上不少。訓練準確度和測試準確度之間的這種差距是「過擬和( over fitting )」所造成的，而這是Ch3的主題。


## 2.2 神經網路的數據表示
前面所示用的數據存儲在多維度的Numpy數組中，亦即張量( tensor )；其核心概念在於，他是矩陣像任意維度的推廣，他是一個數據容器，包含的數據乎總是數值數據，必須注意的是，張量的維度通常叫做「軸( axis )」。


### 2.2.1 標量( 0D 張量 )
僅包含一個數字的張量叫做標量( scalar )，在Numpy中，一個float32或是float64的數字就是一個標量張量( 或稱標量數組 )。若想要確認Numpy張量軸的個數(也稱為「階」rank)，我們可以用ndim屬性來查看。


In [31]:
'''
向量有5個元素，就可以稱為5D( Dimension )向量
要注意的是，5D向量只有一個軸，沿著軸有5個維度
'''
import numpy as np
x = np.array([12, 3, 6, 12,6])
print(x.ndim)

'''
維度可以表示沿著某個軸上的元素個數
也可以表示張量中軸的各數
這有時讓人非常混亂
因此技術上來說
後者可以被稱為5階張量
意思是張量的階數，亦即軸的個數
'''


1


'\n維度可以表示沿著某個軸上的元素個數\n也可以表示張量中軸的各數\n這有時讓人非常混亂\n因此技術上來說\n後者可以被稱為5階張量\n意思是張量的階數，亦即軸的個數\n'

### 2.2.2 關鍵屬性
張量是由以下關鍵屬性來定義的：
* 軸的個數(「階」( rank ) )<br>
例如，3D張量有3個軸，矩陣有2個軸，這在numpy也較張量的ndim。
* 形狀<br>
這是一個整數元祖表示張量沿著每個軸所張開的大小，例如，矩陣的形狀為(3,5)，向量的形狀為(5,)，標量的形狀為空，亦即()。
* 數據類型<br>
也就是張量所包數據的類型，例如，張量的類型可以是float32、unit8、float64等。


In [2]:
'''
為了具體說明2.2.2內容，我們載入MNIST數據集，如下
這裡，train_images事一個由8為整數組成的3階張量
換句話說，就是6000個，每個28x28矩陣組成的數組
每個這樣的矩陣都是一張灰階圖像，元素取值範圍為0-255
'''

from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print('軸的個數，(.ndim) = ', train_images.ndim)
print('形狀(.shape) = ', train_images.shape)
print('數據的類型(.shape) = ', train_images.dtype)


Using TensorFlow backend.


軸的個數，(.ndim) =  3
形狀(.shape) =  (60000, 28, 28)
數據的類型(.shape) =  uint8


In [3]:
# 2-7 顯示第599個數字
digit = train_images[599]

import matplotlib.pyplot as plt
plt.imshow(digit, cmap=plt.cm.binary)
plt.show()


<Figure size 640x480 with 1 Axes>

### 2.2.3 在Numpy中操作張量
在前面的例子中，我們使用語法train_images[i]來選擇「沿著第一個軸」的特定數字。其中，選擇張量的特定元素叫做「張量切片( sensor slicing )」。Numpy數組上的張量切片運算舉例如下：<br><br>

下面這個例子選擇第10~99=90個數字，並將其放在形狀為(90, 28, 28)的數組中。


In [6]:
my_slice = train_images[10:100]
print(my_slice.shape)

# 等同寫法1
# my_slice = train_images[10:100, :, :]

# 等同寫法2
# my_slice = train_images[10:100, 0:28, 0:28]

'''
一般來說，我們沿著每個張量軸在任意兩個索引值之間進行選擇
例如，我們可以在所有圖像的右下角選出14x14像素的區域：
my_slice = train_images[:, 14:, 14:]
'''


(90, 28, 28)


3

### 2.2.4 數據批量的概念
通常DL中所有數據張量的第一個軸都是「樣本軸」，而DL模型不會同時處理整個數據集，而是將數據拆分成小批量，具體說明如下。


In [9]:

###
# 下面是MNIST數據集的第一個批量，批量大小為128
###

batch = train_images[:128]

# 然後是下一批量

batch = train_images[128 : 256]

# 然後是第n個批量

n=99
batch = train_images[128 * n : 128 *(n+1)]

###
# 對於這種批量張量，第一個軸(0軸)叫做批量軸( batch axis )
# 這是在使用Keras和其他DL庫時，會經常碰到的術語
###

###
## 我們來看看幾個常見的數據張量 ##
# 向量數據：2D張量、形狀為( samples, features )
# 時間序列數據或序列數據：3D張量、形狀為( samples, timesteps, features )
# 圖像：4D張量、形狀為( samples, height, width, channels )
# 影片：5D張量、形狀為( samples, frames, height, width, channels )
###


## 2.3 神經網路的"齒輪"，張量運算
所有電腦程式最終都可以簡化成為二進制輸入上的一些二進制運算( AND、OR、NOR、etc )，與此類似，深度NN學到的所有變換也都可以簡化成數值數據張量上的一些張量運算( tensor operation )，例如加上張量、乘以張量等運算。<br><br>

在最開始的例子中，我們通過Dense層來建構NN，如下：<br><br>

keras.layers.Dense(512, activation='relu')<br><br>

這個層可以理解為，輸入一個2D張量，返回另一個2D張量，亦即輸入張量的新表示，如下。<br><br>

output = relu (dot(W, input) + b)<br><br>

我們將上式拆解，得知有三個張量運算：輸入張量input和張量W之間的點積運算(dot)、得到的2D張量與向量b之間的廣播運算(+)、最後的relu運算。


### 2.3.1 逐元素運算
relu運算和加法都是逐元素( element-wise )的運算，即該運算獨立第應用於張量豬的每個元素，也就是說這很適合應用於向量化的實現，如下。<br><br>


In [12]:
### 
# 當然，relu也能使用for陳述式來實現，如下
###

def naive_relu(x):
    assert len(x.shape) == 2 # x是一個Numpy的2D張量
    
    x = x.copy() # 避免覆蓋輸入張量
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i,j] = max(x[i,j], 0) # relu(x) 是 max(x, 0)
        return x

### 
# 對於廣播(加法)採用同樣的實現方法
###

def naive_add(x ,y):           # |
    assert len(x.shape) == 2    # | x & y 是Numpy的2D張量
    assert x.shape == y.shape   #<-
    
    x = x.copy()                    #<-
                                    # | 避免覆蓋輸入張量
    for i in range(x.shape[0]):     # |
        for j in range(x.shape[1]): # | 
            x[i,j] += y[i,j]        
        return x
    
### 
# 根據同樣的方法，我們可以實現逐元素的乘法、減法等運算。
# 這些運算都是優化好的Numpy內置函數，
# 由基礎線性代數子程序(BLAS, basic linear algebra subprograms)實現。
# 通常用Fortran、C語言來實現，因此，在Numpy中可以直接進行下列逐元素運算。
###

### 
# import numpy as np
# z = x + y                 <==逐元素的相加
# z = np.maximum(z, 0.)     <==逐元素的relu
###


### 2.3.2 廣播
形狀上，較小的張量會被「廣播( broadcast )」，以匹配較大張量的形狀，其中，廣播包含以下兩步驟：<br>
*(1)向較小的張量添加軸(叫做「廣播軸」)，使其ndim與較大的張量相同
*(2)較小的張量沿著新軸重複進行廣播的動作，直到其形狀與較大的張量相同<br><br>

具體過程如下例：<br>
x.ndim = (32, 10)<br>
y.ndim = (10,)<br>
我們給y添加空的第一個「廣播軸」，這樣y的形狀就變成(1,10)，再將y軸沿著新軸重複32次，得到y的新張量形狀為(32,10)。<br>
重複的做法為，Y[i, :] == y for i in range(0, 32)。<br>
這種重複的作法在實際上並不會直接創建一個新的2D張量，因為這樣做非常低效，實際上他只會出現在算法中，而並不會發生在內存中，代碼如下。


In [ ]:
def naive_add_matrix_and_vector(x,y):
    assert len(x.shape) == 2        # x 是 Numpy的2D張量
    assert len(y.shpae) == 1        # y 是 Numpy的向量
    assert x.shape[1] == y.shpae[0]
    
    x = x.copy()                    # 避免覆蓋輸入張量
    
    for i in range(x.shape[0]):
        for j in range(y[1]):
            x[i, j] += y[j]
        return x
###
# 上例只是簡單的廣播算法實現
# 如果一個張量的形狀是(a, b, ..., n, n+1, ..., m)
# 另一個張量的形狀是(n, n+1, ..., m)
# 那麼通常可以利用廣播對他們做兩個張量之間的逐元素運算
# 下面我們來看看
# 利用廣播將逐元素的maximum運算應用於兩個形狀不同的張量
###

import numpy as np
x = np.random.random((64, 3, 32, 10))       # x的形狀為(64, 3, 32, 10)的隨機張量
y = np.random.random((32, 10))              # y的形狀為(32, 10)的隨機張量
z = np.maximum(x, y)                        # 輸出z的形狀是(64, 3, 32, 10)，與x相同


### 2.3.3 張量點積
張量積，與逐元素的乘積不同，他將輸入張量與元素合併在一起。例如，z = np.dot(x, y)。


In [ ]:
###
# 從數學的角度來看，我們來看看點積運算做了甚麼
###

def naive_vector_dot(x, y):
    assert len(x.shape) == 1    # | x & y都是Numpy向量
    assert len(y.shape) == 1    # |
    assert x.shape[0] == y.shape[1]
    
    z = 0.
    for i in range(x.shape[0]):
        z += x[i] * y[i]
        return z                # 返回的z僅是一個標量

###
# 注意，兩個向量之間的點積是一個標量
# 而且只有元素個數相同的向量之間才能做點積
# 依此類推，對兩個矩陣(2D張量)做點積，返回值是一個向量(1D張量)
# 其中每個元素是y和x的每一行之間的點積，其實現過程如下
###


In [ ]:
import numpy as np
def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2            # | x 是Numpy矩陣
    assert len(x.shape) == 1            # | y 是Numpy向量
    assert x.shape[1] == y.shape[0]     # | x的第一維和y的第0維大小必須相同
    
    z = np.zeros((x.shape[0]))          # <-
                                        #  |
    for i in range(x.shape[0]):         #  | 
        for j in range(x.shape[1]):     #  | 這個運算返回一個全是0的向量
            z[i] += x[i,j] * y[j]       #  | 其形狀與x.shape[0]相同
        return z
    
###
# 另外，我們可以從之前寫過的代碼
# 作為對照，從中可以看出
# 矩陣-向量點積與向量點積之間的關係
###

def naive_matrix_vector_dot(x, y):
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i, :], y) 
        
###
# 注意，只要兩張張量中其中一個張量的ndim > 1
# 那麼dot運算就不再會是對稱的，可以用紙筆試算看看
# 也就是說 dot(x,y) != dot(y,x)
###

###
# 當然，點積可以推到任意各軸的張量
# 對於兩個矩陣x和y，若且為若x.shape[1]==y.shape[0]時
# 我們才可以對她們做點積(dot(x,y))
# 得到的結果試另一個形狀的點積
# (x.shape[0], y.shape[1])的矩陣
# 亦即，元素為x的列與y的行之間的點積
# 簡單實行如下
###

def naive_martix_dot(x, y):
    assert len(x.shape) == 2            #  x 是Numpy矩陣
    assert len(y.shape) == 2            #  y 是Numpy矩陣
    assert x.shape[1] == y.shape[0]     #  x 的行與y的列維度大小必須相同
    
    z = np.zeros((x.shape[0], y.shape[1]))  # 這個運算返回特定形狀的零矩陣
    for i in range(x.shape[0]):             # 歷遍所有x的列
        for j in range(y.shape[1]):         # 歷遍所有y的行
            row_x = x[i, :]
            column_y = y[:, j]
            z[i, j] = naive_vector_dot(row_x, column_y)
        return z


### 解說
為了方便理解點積的形狀匹配，可以將輸入張量和輸出張量像fig 2.5這樣排列，利用可視化來幫助理解。<br>
![2.3.3 圖解矩陣點積](./img/2.3.3.PNG)<br><br>

加以推廣，只要對更高維的張量做點積，只要形狀匹配遵循與前面2D張量相同的原則：
* (a, b, c, d) . (d) - - > (a, b, c) 
* (a, b, c, d) . (d, e) - - > (a, b, c, e)


### 2.3.4 張量變形
第三個重要的張量運算式張量變形( tensor reshaping )。我們通常在預處理時用到了這個運算，也就是將圖像數據輸入神經網路之前。<br><br>

張量變形是指改變張量的列與行，以得到想要的形狀。但要注意的是，變形後的張量元素總個數與初始張量相同，舉例如下。


In [22]:
import numpy as np
x = np.array([[0., 1.],
             [2., 3.],
             [4., 5.]])     # 3列2行
print(x.shape)
x = x.reshape((6, 1))       # 6列1行
print('\n',x)
x = x.reshape(2,3)          # | 2列3行
                            # | 其效果同x = np.transpose(x)
print(x)


(3, 2)

 [[0.]
 [1.]
 [2.]
 [3.]
 [4.]
 [5.]]
[[0. 1. 2.]
 [3. 4. 5.]]


### 2.3.5 深度學習的幾何解釋
神經網路完全由一系列張量運算所構成的，而這些張量運算都只是輸入數據的幾何變換。因此，我們可以想像神經網路為，在高維空間中非常複雜的幾何變換，這種變換可以通過許多簡單的步驟來實現。<br><br>
其實，深度學習將複雜的幾何變換逐步分解為一長串基本的幾何變換，這與人類展開一坨由兩張不同顏色柔在一起所形成的紙球如出一轍。DNN的每一層都通過變換使數據解開一點點，許多層堆疊在一起，可以實現非常複雜的解開過程。如下圖所示。<br>
![fig2.9 解開複雜的數據流形](./img/2.3.5.PNG)<br>


### 2.4 神經網路的"引擎"：基於梯度的優化
上一節介紹過，我們在第一個神經網路示例中，每個NN都用下述方法進對輸入數據進行變換。<br>
output = relu(dot(W, input) + b)<br>
在這個範例中，W、b都是張量，均為該層的「屬性」，分別對應kernael和bias屬性。這些屬性包含網路從觀察訓練數據中學到的訊息。<br><br>

一開始，這些權重矩陣取較小的隨機值，這一步叫做隨機初始化( random initialization )。根據初始的隨機值調整，下一步是根據反饋信號逐漸調節這些權重，這個逐漸調節的過程叫做「訓練」，也是ML中的「學習」。<br><br>

上述的過程發生在一個「訓練循環( training loop)內」，必要時這個過程會一直重複，具體展示如下。
* (1)抽取訓練樣本x和對應目標y的組成的數據批量。
* (2)在x上運行網路[這個步驟稱為「前向傳播( forwawrd pass )]，得到預測值y_pred。
* (3)計算網路在這批數據上的損失，用於衡量y_pred與真實值y之間的差距。
* (4)更新網路的所有權重，使網路在這批數據上的損失略為下降。<br>
以上乍看像就像魔法一般，但將其簡化成基本步驟，那麼會變得更容易理解。<br><br>

第一步只是輸入/輸出(I/O)的代碼，第二步和第三步僅僅是一些張量運算的應用，所以我們完全可以利用上一節所學到的知識來實現這兩部，困難點在於第四步，若考慮網路中的某個權重參數，該如何得知更新時要變大還是變小，變化多少也是一個問題?<br><br>

若要進行高效率的計算，就需要引入「可微分、梯度」的概念了。


### 2.4.1 甚麼是導數
假設有一個連續的光華函數f(x) = y，將實數x映射為另一個實數y。由於x是**連續的**，x的微小變化(epsilon_x)只能導致y(epsilon_y)的微小變化---這就是函數連續性的直觀解釋：<br>
f(x + epsilon_x) = y + epsilon_y<br>
此外，由於函數是**光滑的**，在某個點p附近，如果epsilon_x足夠小(趨近於0)，就可以將f近似為斜率為a的線性函數，這樣epsilon_y就變成了a  epsilon_x:<br>
f(x + epsilon_x) = y + a * epsilon_x<br>
顯然，只有在x足夠接近p時，這個線性近似才是合法的，如下圖所示。<br>
![f在p點的導數](./img/2.4.1.PNG)<br>


### 2.4.2 梯度
梯度( gradient )是張量運算的導數，他是導數這一概念向多元函數導數的推廣，多元函數是以張量作為輸入的函數。<br>
假設我們有：
* 輸入向量 x
* 權重參數(矩陣形狀) W
* 一個目標直 y
* 一個損失函數 loss <br>
我們可以用W來計算預測值y_pred，然後計算損失(也就是計算y_pred與y之間的距離)。<br>
y_pred = dot(W, x) <br>
loss_value = loss(y_pred, y) = f(W) <br>
如果輸入數據x和y保持不變，那麼這可以看做將W映射到損失函值的函數。 <br><br>


### 2.4.3 隨機梯度下降
基於2.4節開頭總結的四步驟學習算法：當前在隨機數據批量上的損失，一點一點地對參數進行調節。由於處理的是一個可微分函數，我們可以計算出他的梯度，從而有效率地執行第四步。沿著梯度的反方向更新權重，損失每次都會變小一點。<br>
* (1)抽取訓練樣本x和對應目標y的組成的數據批量。
* (2)在x上運行網路[這個步驟稱為「前向傳播( forwawrd pass )]，得到預測值y_pred。
* (3)計算網路在這批數據上的損失，用於衡量y_pred與真實值y之間的差距。
* (4)計算損失相對於網路權重參數的梯度[一次「反向傳播( backward pass )]。
* (5)將參數沿著梯度的反方向移動一點，比如 W -= step * gradient ，從而使這批數據上的損失小一點 <br><br>

以上所描述的方法稱做「小批量隨機梯度下降( mini-batch stochastic gradient descent, SGD )」。一般來說，SGD對於深度學習的術語中有以下名詞：
* 小批量SGD --> 一次只抽取一個樣本和目標。
* 真SGD    --> 一次抽取一批數據。
* 批量SGD   --> 每一次迭代都在所有數據上運行。<br><br>

其中，SGD有很多變體，最值得關注的變體是「動量」，動量解決了SGD的兩個問題：(1).收斂速度、(2).局部最小值。我們來簡單實現其程式碼。


In [23]:
past_velocity = 0.
momentum = 0.1      # 不變的動量因子
while loss > 0.01:  # 優化循環
    w, loss, gradient = get_current_parameters()
    velocity = past_velocity * momentum - learning_rate * gradient
    w = w + momentum * velocity - learning_rate * gradient
    past_velocity = velocity
    update_parameter(w)
    

NameError: name 'loss' is not defined

### 2.4.4 鏈式求導：反向傳播算法
我們假設有一個網路f包含3個張量運算，a、b、c，還有3個權重矩陣W1、W2、W3，以式子表達如：f(W1, W2, W3) = a(W1, b(W2, c(W3)))。<br><br>

根據微積分的知識，我們可以知道鏈式法則( chain rule )： <br>$ (f(g(x)))' = f'(g(x)) * g'(x) $<br>
這種由函數最外層向內微分的算法稱為「反向傳播( backpropagation )」，有時候也稱為「反式微分( reverse-mode differentiation )」。



## 2.5 回顧第一個例子
根據前面三節所學，我們來重新閱讀以下示例中的每一段代碼，下面是輸入數據；<br>
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()<br><br>

train_images = train_images.reshape((60000, 28 * 28))<br>
train_images = train_images.astype('float32') / 255<br><br>

test_images = test_images.reshape((10000, 28 * 28))<br>
test_images = test_images.astype('float32') / 255<br>

現在我們明白了，輸入圖像保存在float32格式的Numpy張量中，形狀分別為(60000, 784)、(10000, 784)。<br><br>

下面是建構神經網路：<br>
network = models.Sequential()<br>
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))<br>
network.add(layers.Dense(10, activation='softmax'))<br><br>

這個網路包含兩個Dense層，每層都對輸入數據進行一些簡單的張量運算，這些運算都包含權重張量。權重張量式該層的屬性，裡面保存了網路所學到的「知識( knowledge )」。<br><br>

下面是網路的編譯：<br>
network.compile(optimizer='rmsprop', <br>
                loss='categorical_crossentropy', <br>
                metrics=['accuracy']) <br><br>
                
categorical_crossentropy是損失函數，用於學習權重張量的反饋訊號，在「訓練階段」應使他最小化，而這個減小損失的動作是通過「小批量SGD」來實現的。梯度下降的具體方法由第一個參數設定，亦即rmsprop優化器。<br><br>

最後，下面式訓練循環：<br>
network.fit(train_images, train_labels, epochs=5, batch_size=128)<br><br>

現在，我們明白在調用fit時發生了甚麼：網路開始在訓練數據上進行迭代時，我們指定每個批量為128個樣本，共迭代5次[在所有訓練數據上迭代一次稱為一個**輪次( epech )**]。在每次迭代過程中，網路會計算「批量數據的損失」相對於權重的梯度，並相應地更新權重，使得網路能夠以很高的準確度對手寫數字進行分類。


# 本章小結
* **學習**是指找到一組模型參數，使得在給定的訓練數據樣本和相對應目標值上的損失函數進行最小化過程。
* 學習的過程：隨機選取包含數據樣本及其目標值的批量，計算批量損失相對於網路參數的梯度。隨後將網路參數沿著梯度的「反方向」稍稍移動( 移動距離由學習率來指定 )。
* 整個學習過程之所以能夠實現，是因為NN是一系列可微分的張量運算，因此可以利用求導數的鏈式法則( chain rule )來得到梯度函數，這個函數將當前參數(W)和當前數據批量(X)映射為一個梯度值。
* 後續幾張我們會經常遇到兩個關鍵的觀念：**損失( loss )**、**優化器( optimizer )**。將數據輸入網路之前，我們需要先定義上述兩者。
* **損失( loss )**，是在訓練過程中需要最小化的量，因此，此量能夠衡量當前任務是否正往正確的方向前進。
* **優化器( optimizer )**，是使用損失梯度更新參數的具體方式，比如RMSProp優化器、帶動量的隨機梯度下降( Momentum )等。